In [1]:
import maelzel.partialtracking as pt
import pitchtools.vectorized as ptv
import csoundengine as ce
from sndfileio import *
import bpf4

In [2]:
samples, sr = sndread("../snd/colours-german-male.flac")
sp = pt.analyze(samples, sr=sr, resolution=50, windowsize=60, hoptime=1/120)
sp

Spectrum(numpartials=10082, start=0.014, end=10.729, density=6351.3)

In [3]:
synth = sp.play()
synth




--Csound version 6.19 (double samples) Jan 13 2024
[commit: c53556fbbb69d14554a4a4bd91efa441a1701d70]
libsndfile-1.2.0
sr = 44100.0, kr = 689.062, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
audio buffered in 256 sample-frame blocks
reading 2048-byte blks of shorts from adc:Built-in Audio Pro: (RAW)
writing 512 sample blks of 64-bit floats to dac:Built-in Audio Pro:
SECTION 1:


Button(description='Stop', style=ButtonStyle())

Output()

Synth(▶ .playPartials=501.0002 start=0.446 dur=-1.000 p4=104 ifn=0 iskip=3065 inumrows=853 inumcols=1 kspeed=0 kloop=0 kminfreq=0 kmaxfreq=6 iflags=0 istart=0 istop=1 kfreqscale=1 ichan=1 kbwscale=1 kgain=0 iposition=0 kfreqoffset=0 kminbw=1 kmaxbw=0)

Playback the voiced and unvoiced parts of the spectrum separately. 

In [4]:
# Use the default session
session = synth.session

# Or create a new session
# session = ce.Session()

In [5]:
speed = 0.25
with session as s:
    sp.play(speed=speed, freqoffset=0, freqscale=1, maxbw=0.01, gain=1, chan=1, session=session)
    sp.play(speed=speed, freqoffset=0, freqscale=1, minbw=0.01, gain=20, minfreq=2000, chan=2, session=session, bwscale=5)
s

Button(description='Stop Synths', style=ButtonStyle())

Output()

Session(maelzel, synths=3)

In [29]:


def quantize(partial, simp=0.4, steps=2):
    """quantize and simplify partial"""
    
    if simp > 0:
        p2 = partial.simplified(simp)
    else:
        p2 = partial.copy()
    pitches = (ptv.f2m(p2.freqs) / steps).round() * steps
    p2.data[:,1] = ptv.m2f(pitches)
    return p2

In [36]:
quantizedspec = pt.Spectrum([quantize(p, simp=0.2, steps=4) for p in sp.partials])
quantizedspec

Spectrum(numpartials=10082, start=0.014, end=10.729, density=2126.3)

In [34]:
%matplotlib qt
quantizedspec.plot()

QSocketNotifier: Can only be used with threads started with QThread


In [37]:
quantizedspec.play(maxbw=0.2)

Button(description='Stop', style=ButtonStyle())

Output()

Synth(𝍪 .playPartials=501.002 start=1197.174 dur=-1.000 p4=112 ifn=0 iskip=604 inumrows=1243 inumcols=1 kspeed=0 kloop=0 kminfreq=0 kmaxfreq=6 iflags=0 istart=0 istop=1 kfreqscale=1 ichan=1 kbwscale=1 kgain=0 iposition=0 kfreqoffset=0 kminbw=0.2 kmaxbw=0)

In [3]:
import numpyx
from maelzel.music import scale

def quantizeToScale(partial, steps=(0, 2, 2, 1, 2, 2, 2, 1)):
    pitches = scale.pitchscale("C0", steps=steps)
    scaleFreqs = ptv.m2f(pitches)
    return partial.freqTransform(lambda freqs: numpyx.nearestitem(scaleFreqs, freqs))
    

In [14]:
quantizedspec2 = pt.Spectrum([quantizeToScale(p.simplified(0.3), steps=(0, 2, 1, 3, 1, 2, 2, 1)) for p in sp.partials])
quantizedspec2

Spectrum(numpartials=10082, start=0.014, end=10.729, density=2003.2)

In [15]:
quantizedspec2.play(speed=0.25)

Button(description='Stop', style=ButtonStyle())

Output()

Synth(𝍪 .playPartials=501.0006 start=199.041 dur=-1.000 p4=107 ifn=0 iskip=510 inumrows=1303 inumcols=0.25 kspeed=0 kloop=0 kminfreq=0 kmaxfreq=6 iflags=0 istart=0 istop=1 kfreqscale=1 ichan=1 kbwscale=1 kgain=0 iposition=0 kfreqoffset=0 kminbw=1 kmaxbw=0)

In [ ]:
quanti